In [4]:
# Instalar bibliotecas necessárias
!pip install transformers datasets torch accelerate


In [10]:

# 1. Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Caminhos dos arquivos
caminho_entrada = '/content/drive/MyDrive/Doutorado Unesp/IA/projetoLLM/palavras.json'
caminho_saida = '/content/drive/MyDrive/Doutorado Unesp/IA/projetoLLM/dados_formatados.json'
caminho_modelo_final = '/content/drive/MyDrive/Doutorado Unesp/IA/projetoLLM/guarani-pt-model'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
with open(caminho_entrada, 'r', encoding='utf-8') as f:
    dados = json.load(f)

print(f"Total de itens carregados: {len(dados)}")
print("Primeiro item:")
print(dados[0] if len(dados) > 0 else "JSON vazio")


NameError: name 'json' is not defined

In [ ]:
# 3. Importar bibliotecas
import json
import torch
from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForCausalLM,
    Trainer, TrainingArguments, DataCollatorForLanguageModeling,
    GPT2Tokenizer, GPT2LMHeadModel
)
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.model_selection import train_test_split

# 4. Carregar e processar JSON original
print("Carregando dados...")
with open(caminho_entrada, 'r', encoding='utf-8') as f:
    dados = json.load(f)

print(f"Total de registros encontrados: {len(dados)}")

# 5. Formatar para prompt/completion
formatted_data = []
for item in dados["frases"]:  # <-- ajuste aqui
    if 'guarani' in item and 'portugues' in item:
        formatted_data.append({
            "text": f"Traduza do guarani para o português: {item['guarani']} -> {item['portugues']}"
        })
print(f"Total de exemplos formatados: {len(formatted_data)}")

# 6. Dividir dados em treino e validação
train_data, val_data = train_test_split(formatted_data, test_size=0.1, random_state=42)
print(f"Dados de treino: {len(train_data)}, Validação: {len(val_data)}")



In [ ]:
# 7. Salvar dados formatados
with open(caminho_saida, 'w', encoding='utf-8') as f_out:
    json.dump({
        'train': train_data,
        'validation': val_data,
        'total': len(formatted_data)
    }, f_out, ensure_ascii=False, indent=2)

# 8. Criar Datasets HuggingFace
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# 9. Configurar tokenizer
print("Configurando tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Adicionar tokens especiais para melhor performance
special_tokens = {
    "additional_special_tokens": ["<GUARANI>", "<PORTUGUÊS>", "<TRADUÇÃO>"]
}
tokenizer.add_special_tokens(special_tokens)

# 10. Função de tokenização otimizada
def tokenize_function(examples):
    # Usar comprimento dinâmico baseado nos dados
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,  # Aumentado para acomodar frases mais longas
        return_special_tokens_mask=True
    )

print("Tokenizando dados...")
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# 11. Configurar modelo com arquitetura otimizada
print("Configurando modelo...")
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_layer=6,        # Aumentado para melhor capacidade
    n_embd=384,       # Embedding maior
    n_head=12,        # Mais cabeças de atenção
    n_positions=128,  # Sequências mais longas
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

model = GPT2LMHeadModel(config)
model.resize_token_embeddings(len(tokenizer))

print(f"Modelo criado com {model.num_parameters():,} parâmetros")

# 12. Data collator otimizado
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    return_tensors="pt"
)



In [ ]:
# 13. Argumentos de treinamento otimizados
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import Trainer,TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
)


# 14. Função de métricas customizada
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Calcular perplexidade
    shift_logits = predictions[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    perplexity = torch.exp(loss)
    return {"perplexity": perplexity.item()}

# 15. Criar e configurar Trainer
print("Iniciando treinamento...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 16. Treinar modelo
trainer.train()



In [ ]:
# 17. Salvar modelo final
print("Salvando modelo...")
model.save_pretrained(caminho_modelo_final)
tokenizer.save_pretrained(caminho_modelo_final)

# Salvar também localmente
model.save_pretrained("./guarani-pt-model-final")
tokenizer.save_pretrained("./guarani-pt-model-final")

print("Treinamento concluído!")

# 18. Função de tradução melhorada
def traduzir(termo_guarani, modelo=model, tokenizer=tokenizer, max_tentativas=3):
    """
    Traduz termo do guarani para português
    """
    resultados = []

    # Diferentes formatos de prompt
    prompts = [
        f"Traduza do guarani para o português: {termo_guarani} ->",
        f"Guarani: {termo_guarani} | Português:",
        f"{termo_guarani} significa"
    ]

    modelo.eval()
    with torch.no_grad():
        for prompt in prompts:
            try:
                inputs = tokenizer(prompt, return_tensors="pt", padding=True)

                outputs = modelo.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_length=inputs.input_ids.shape[1] + 20,
                    num_return_sequences=1,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id,
                    no_repeat_ngram_size=2,
                    early_stopping=True
                )

                resultado = tokenizer.decode(outputs[0], skip_special_tokens=True)
                # Extrair apenas a tradução
                if "->" in resultado:
                    traducao = resultado.split("->")[-1].strip()
                elif "|" in resultado:
                    traducao = resultado.split("|")[-1].replace("Português:", "").strip()
                else:
                    traducao = resultado.replace(prompt, "").strip()

                if traducao and len(traducao) > 1:
                    resultados.append(traducao)

            except Exception as e:
                print(f"Erro no prompt '{prompt}': {e}")
                continue

    # Retornar a melhor tradução (mais comum ou primeira válida)
    if resultados:
        return resultados[0]
    else:
        return "Tradução não encontrada"

# 19. Função de avaliação do modelo
def avaliar_modelo(exemplos_teste=None):
    """
    Avalia o modelo com exemplos de teste
    """
    if exemplos_teste is None:
        # Usar alguns exemplos dos dados de validação
        exemplos_teste = [
            ("mba'e", "que/o que"),
            ("ava", "homem"),
            ("kuña", "mulher"),
            ("tembi'u", "comida"),
            ("y", "água")
        ]

    print("=== AVALIAÇÃO DO MODELO ===")
    acertos = 0
    total = len(exemplos_teste)

    for guarani, esperado in exemplos_teste:
        traducao = traduzir(guarani)
        correto = any(palavra in traducao.lower() for palavra in esperado.lower().split('/'))

        print(f"Guarani: {guarani}")
        print(f"Esperado: {esperado}")
        print(f"Obtido: {traducao}")
        print(f"Correto: {'✓' if correto else '✗'}")
        print("-" * 40)

        if correto:
            acertos += 1

    precisao = (acertos / total) * 100
    print(f"\nPrecisão: {acertos}/{total} ({precisao:.1f}%)")
    return precisao

# 20. Executar testes
print("\n" + "="*50)
print("TESTANDO MODELO TREINADO")
print("="*50)

# Exemplos de teste
exemplos_teste = [
    "mba'e",      # o que
    "pe",         # você
    "ha'e",       # ele/ela
    "ava",        # homem
    "tembi'u",    # comida
]

for exemplo in exemplos_teste:
    resultado = traduzir(exemplo)
    print(f"'{exemplo}' -> '{resultado}'")

# Avaliar modelo se houver dados de teste
try:
    precisao = avaliar_modelo()
    print(f"\nModelo salvo com precisão de {precisao:.1f}%")
except:
    print("\nModelo salvo com sucesso!")

print(f"\nModelo final salvo em: {caminho_modelo_final}")